In [1]:
import pandas as pd
from nlpia.data.loaders import get_data
pd.options.display.width=120
sms=get_data('sms-spam')
index=['sms{}{}'.format(i,'!'*j) for (i,j) in zip(range(len(sms)),sms.spam)]

INFO:nlpia.constants:Starting logger in nlpia.constants...
INFO:nlpia.loaders:No BIGDATA index found in C:\ProgramData\Anaconda3\lib\site-packages\nlpia\data\bigdata_info.csv so copy C:\ProgramData\Anaconda3\lib\site-packages\nlpia\data\bigdata_info.latest.csv to C:\ProgramData\Anaconda3\lib\site-packages\nlpia\data\bigdata_info.csv if you want to "freeze" it.
INFO:nlpia.futil:Reading CSV with `read_csv(*('C:\\ProgramData\\Anaconda3\\lib\\site-packages\\nlpia\\data\\mavis-batey-greetings.csv',), **{'low_memory': False})`...
INFO:nlpia.futil:Reading CSV with `read_csv(*('C:\\ProgramData\\Anaconda3\\lib\\site-packages\\nlpia\\data\\sms-spam.csv',), **{'low_memory': False})`...
INFO:nlpia.futil:Reading CSV with `read_csv(*('C:\\ProgramData\\Anaconda3\\lib\\site-packages\\nlpia\\data\\sms-spam.csv',), **{'nrows': None, 'low_memory': False})`...


In [2]:
sms=pd.DataFrame(sms.values,columns=sms.columns,index=index)

In [4]:
len(sms)

4837

In [5]:
sms.spam.sum()

638

In [6]:
sms.head(6)

,spam,text
sms0,0,"Go until jurong point, crazy.. Available only ..."
sms1,0,Ok lar... Joking wif u oni...
sms2!,1,Free entry in 2 a wkly comp to win FA Cup fina...
sms3,0,U dun say so early hor... U c already then say...
sms4,0,"Nah I don't think he goes to usf, he lives aro..."
sms5!,1,FreeMsg Hey there darling it's been 3 week's n...


In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [8]:
from nltk.tokenize.casual import casual_tokenize

In [9]:
tfidf_model=TfidfVectorizer(tokenizer=casual_tokenize)

In [10]:
tfidf_docs=tfidf_model.fit_transform(raw_documents=sms.text).toarray()

In [14]:
tfidf_docs.shape

(4837, 9232)

In [15]:
sms.spam.sum()

638

In [16]:
mask=sms.spam.astype(bool).values

In [18]:
spam_centroid=tfidf_docs[mask].mean(axis=0)

In [19]:
ham_centroid=tfidf_docs[~mask].mean(axis=0)

In [20]:
spam_centroid.round(2)

array([0.06, 0.  , 0.  , ..., 0.  , 0.  , 0.  ])

In [21]:
ham_centroid.round(2)

array([0.02, 0.01, 0.  , ..., 0.  , 0.  , 0.  ])

In [23]:
spamminess_score=tfidf_docs.dot(spam_centroid-ham_centroid)

In [24]:
# fitting into range (0,1)

In [25]:
from sklearn.preprocessing import MinMaxScaler

In [27]:
sms['lda_score']=MinMaxScaler().fit_transform(spamminess_score.reshape(-1,1))

In [32]:
sms['lda_predict']=(sms.lda_score>.5).astype(int)

In [34]:
sms['lda_predict'].head()

sms0     0
sms1     0
sms2!    1
sms3     0
sms4     0
Name: lda_predict, dtype: int32

In [35]:
sms['spam lda_predict lda_score'.split()].round(2).head(6)

,spam,lda_predict,lda_score
sms0,0,0,0.23
sms1,0,0,0.18
sms2!,1,1,0.72
sms3,0,0,0.18
sms4,0,0,0.29
sms5!,1,1,0.55


In [36]:
# accuracy

In [40]:
round((1.-(sms.spam-sms.lda_predict).abs().sum()/len(sms)),4)

0.9775